In [ ]:
import numpy as np
import pandas as pd


dataset = pd.read_csv("path_to_file.csv",encoding='cp1250', sep=";")
drop_columns=["city_id","currency_code","timezone","flow","request_timestamp_utc","begintrip_timestamp_utc","dropoff_timestamp_utc",
              "promotion_usd","credits_usd","driver_upfront_fare_usd","original_fare_usd","base_fare_usd","surge_fare_usd","minimum_fare_roundup_usd",
              "per_mile_fare_usd","per_minute_fare_usd","cancellation_fee_usd","rounding_down_amount_usd","service_fee_usd","toll_amount_usd",
              "booking_fee_usd","earnings_boost_usd","wait_time_fare_usd","long_distance_surcharge_usd","vehicle_trip_number"]

for column in drop_columns:
    dataset.drop(column, axis='columns', inplace=True)
    
dataset.info()


ModuleNotFoundError: No module named 'plotly'

In [ ]:
cancelled_trips = pd.DataFrame(columns=dataset.columns)
cancelled_rows = dataset[dataset['is_completed'].astype(str) == 'False']
cancelled_trips = cancelled_rows.copy()
dataset = dataset[dataset['is_completed'] != False]
 
cancelled_trips.head()  



,product_type_name,global_product_name,request_timestamp_local,begintrip_timestamp_local,dropoff_timestamp_local,begintrip_lat,begintrip_lng,dropoff_lat,dropoff_lng,eta,...,guaranteed_surge_multiplier,ufp_type,cancellation_type,is_directed_dispatch_trip,wait_duration_minutes,driver_trip_number,concierge_source_type,is_scheduled_trip,is_airport_trip,license_plate
89,UberX,UberX,2022-08-18 23:01:08,NaN,NaN,51.25,22.55,51.25,22.55,156.0,...,NaN,ufp_honored,NaN,False,NaN,NaN,NaN,False,False,LUB2035J
120,UberX,UberX,2022-08-20 00:55:19,NaN,NaN,51.25,22.56,51.25,22.56,25.0,...,NaN,ufp_honored,NaN,False,NaN,NaN,NaN,False,False,LUB2035J
134,UberX,UberX,2022-08-20 04:30:34,NaN,NaN,51.23,22.53,51.23,22.52,87.0,...,NaN,ufp_honored,NaN,False,NaN,NaN,NaN,False,False,LUB2035J
154,UberX,UberX,2022-10-12 03:14:37,NaN,NaN,51.22,22.64,51.22,22.70,390.0,...,NaN,ufp_honored,client_standard_cancel,False,NaN,NaN,NaN,False,False,LUB2035J
177,UberX,UberX,2022-10-13 02:25:23,NaN,NaN,51.23,22.54,51.24,22.54,217.0,...,NaN,ufp_honored,client_standard_cancel,False,NaN,NaN,NaN,False,False,LUB2035J


In [ ]:
#time feature engineering

dataset.rename(columns={'eta': 'driver_pickup_eta'}, inplace=True)
    
new_time_features=["request_unix_time","begintrip_unix_time","dropoff_unix_time",
                   "trip_day_of_year","trip_day_of_week","trip_month_of_year","trip_hour_of_day","actual_wait_time"]

for c in new_time_features:
    dataset[c]=0

dataset['request_unix_time'] = pd.to_datetime(dataset['request_timestamp_local']).astype('int64') // 10**9
dataset['begintrip_unix_time'] = pd.to_datetime(dataset['begintrip_timestamp_local']).astype('int64') // 10**9
dataset['dropoff_unix_time'] = pd.to_datetime(dataset['dropoff_timestamp_local']).astype('int64') // 10**9
dataset['trip_day_of_year'] = pd.to_datetime(dataset['begintrip_timestamp_local']).dt.day_of_year
dataset['trip_day_of_week'] = pd.to_datetime(dataset['begintrip_timestamp_local']).dt.day_of_week
dataset['trip_month_of_year'] = pd.to_datetime(dataset['begintrip_timestamp_local']).dt.month
dataset['trip_hour_of_day'] = pd.to_datetime(dataset['begintrip_timestamp_local']).dt.hour    
dataset['actual_wait_time'] = (pd.to_datetime(dataset['begintrip_timestamp_local']) - pd.to_datetime(dataset['request_timestamp_local'])).dt.total_seconds()
dataset['wait_time_delta'] = dataset['actual_wait_time'] - dataset['driver_pickup_eta']

dataset.head()

,product_type_name,global_product_name,request_timestamp_local,begintrip_timestamp_local,dropoff_timestamp_local,begintrip_lat,begintrip_lng,dropoff_lat,dropoff_lng,driver_pickup_eta,...,license_plate,request_unix_time,begintrip_unix_time,dropoff_unix_time,trip_day_of_year,trip_day_of_week,trip_month_of_year,trip_hour_of_day,actual_wait_time,wait_time_delta
0,UberX,UberX,2022-08-09 17:27:57,2022-08-09 17:39:44,2022-08-09 17:54:10,51.21,22.53,51.22,22.46,375.0,...,LUB2035J,1660066077,1660066784,1660067650,221,1,8,17,707.0,332.0
1,UberX,UberX,2022-08-09 21:14:56,2022-08-09 21:17:35,2022-08-09 21:27:30,51.24,22.59,51.24,22.54,69.0,...,LUB2035J,1660079696,1660079855,1660080450,221,1,8,21,159.0,90.0
2,UberX,UberX,2022-08-09 23:08:46,2022-08-09 23:11:24,2022-08-09 23:20:15,51.25,22.56,51.20,22.55,42.0,...,LUB2035J,1660086526,1660086684,1660087215,221,1,8,23,158.0,116.0
3,UberX,UberX,2022-08-09 23:53:09,2022-08-09 23:59:47,2022-08-10 00:02:36,51.25,22.56,51.25,22.55,165.0,...,LUB2035J,1660089189,1660089587,1660089756,221,1,8,23,398.0,233.0
4,UberX,UberX,2022-08-10 03:27:17,2022-08-10 03:40:24,2022-08-10 03:47:15,51.24,22.56,51.28,22.55,327.0,...,LUB2035J,1660102037,1660102824,1660103235,222,2,8,3,787.0,460.0


In [ ]:
# data cleaning/dimension reduction
dataset.drop("status", axis='columns', inplace=True)
dataset.drop("is_completed", axis='columns', inplace=True)
dataset.drop("global_product_name", axis='columns', inplace=True)
dataset.drop("has_destination", axis='columns', inplace=True)
dataset.drop("is_pool_matched", axis='columns', inplace=True)
dataset.drop("is_flat_rate", axis='columns', inplace=True)
dataset.drop("rounding_down_amount_local", axis='columns', inplace=True)
dataset.drop("booking_fee_local", axis='columns', inplace=True)
dataset.drop("toll_amount_local", axis='columns', inplace=True)
dataset.drop("earnings_boost_local", axis='columns', inplace=True)
dataset.drop("driver_cancellation_reason", axis='columns', inplace=True)
dataset.drop("guaranteed_surge_multiplier", axis='columns', inplace=True)
dataset.drop("cancellation_type", axis='columns', inplace=True)
dataset.drop("is_directed_dispatch_trip", axis='columns', inplace=True)

dataset['promotion_local'] = dataset['promotion_local'].replace({'\\N': '0'})
dataset['service_fee_local'] = dataset['service_fee_local'].replace({'\\N': '0'})
dataset['wait_time_fare_local'] = dataset['wait_time_fare_local'].replace({'\\N': '0'})
dataset['long_distance_surcharge_local'] = dataset['long_distance_surcharge_local'].replace({'\\N': '0'})
dataset['wait_duration_minutes'] = dataset['wait_duration_minutes'].replace({'\\N': '0'})
dataset['minimum_fare_roundup_local'] = dataset['minimum_fare_roundup_local'].replace({'\\N': '0'})
dataset['concierge_source_type'] = dataset['concierge_source_type'].replace({'\\N': 'none'})
dataset['credits_local'] = dataset['credits_local'].replace({'\\N': '0'})

dataset['license_plate'] = dataset['license_plate'].str.strip()

if 'pickup_street' in dataset.columns:
    dataset['pickup_street'] = dataset['pickup_street'].str.lower()
    dataset['pickup_city'] = dataset['pickup_city'].str.lower()
    dataset['destination_street'] = dataset['destination_street'].str.lower()
    dataset['destination_city'] = dataset['destination_city'].str.lower()
    




In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(dataset)


,product_type_name,request_timestamp_local,begintrip_timestamp_local,dropoff_timestamp_local,begintrip_lat,begintrip_lng,dropoff_lat,dropoff_lng,driver_pickup_eta,surge_multiplier,is_surged,trip_distance_miles,trip_duration_seconds,is_cash_trip,promotion_local,credits_local,has_driver_upfront_fare,driver_upfront_fare_local,original_fare_local,base_fare_local,surge_fare_local,minimum_fare_roundup_local,per_mile_fare_local,per_minute_fare_local,cancellation_fee_local,service_fee_local,fare_distance_miles,fare_duration_minutes,wait_time_fare_local,service_fee,is_multidestination,driver_surge_multiplier,long_distance_surcharge_local,ufp_type,wait_duration_minutes,driver_trip_number,concierge_source_type,is_scheduled_trip,is_airport_trip,license_plate,request_unix_time,begintrip_unix_time,dropoff_unix_time,trip_day_of_year,trip_day_of_week,trip_month_of_year,trip_hour_of_day,actual_wait_time,wait_time_delta
0,UberX,2022-08-09 17:27:57,2022-08-09 17:39:44,2022-08-09 17:54:10,51.21,22.53,51.22,22.46,375.0,1.0,False,5.267278,832.0,True,-2.14,NaN,False,19.52,19.24,4.40,0.0,NaN,2.655418,0.17,0.0,NaN,5.27,13.866667,0.3,0.19,False,1.0,NaN,ufp_honored,1.983333,161.0,NaN,False,False,LUB2035J,1660066077,1660066784,1660067650,221,1,8,17,707.0,332.0
1,UberX,2022-08-09 21:14:56,2022-08-09 21:17:35,2022-08-09 21:27:30,51.24,22.59,51.24,22.54,69.0,1.0,False,2.969821,595.0,False,-7.01,NaN,False,12.98,7.01,4.40,0.0,NaN,2.655418,0.17,0.0,NaN,2.97,9.916667,NaN,0.19,False,1.0,NaN,ufp_honored,NaN,162.0,NaN,False,False,LUB2035J,1660079696,1660079855,1660080450,221,1,8,21,159.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12390,UberX,2025-04-22 00:43:06,2025-04-22 00:52:44,2025-04-22 01:12:20,51.24,22.49,51.21,22.72,487.0,1.0,False,13.377091,1176.0,False,NaN,NaN,True,48.27,48.27,5.18,0.0,0.0,2.124334,0.20,0.0,-9.65,13.38,19.600000,0.0,0.20,False,1.0,11.49,ufp_honored,0.000000,11998.0,NaN,False,False,LUB5230J,1745282586,1745283164,1745284340,112,1,4,0,578.0,91.0
12391,Green,2025-04-22 01:01:16,2025-04-22 01:18:30,2025-04-22 01:34:34,51.20,22.69,51.22,22.51,714.0,1.0,False,9.581023,964.0,False,-14.87,NaN,True,29.74,29.74,5.18,0.0,0.0,2.124334,0.20,0.0,-5.95,9.58,16.066667,0.0,0.20,False,1.0,0.87,ufp_honored,0.000000,11999.0,NaN,False,False,LUB5230J,1745283676,1745284710,1745285674,112,1,4,1,1034.0,320.0


In [ ]:
for c in dataset.columns:
    print(f"\ncolumn: {c}")
    print(dataset[c].value_counts())


column: product_type_name
product_type_name
UberX             7345
Green             4166
Comfort            167
Uber Pet           129
UberX Priority      31
Connect              1
Name: count, dtype: int64

column: request_timestamp_local
request_timestamp_local
2025-04-22 01:01:16    1
2022-08-09 17:27:57    1
2022-08-09 21:14:56    1
2022-08-09 23:08:46    1
2022-08-09 23:53:09    1
2022-08-10 03:27:17    1
2022-08-10 04:09:53    1
2022-08-11 00:52:37    1
2022-08-11 01:04:25    1
2022-08-11 01:28:27    1
2022-08-11 01:50:36    1
2022-08-11 03:23:16    1
2022-08-11 04:02:07    1
2022-08-11 05:32:41    1
2022-08-11 06:46:38    1
2025-04-17 08:40:02    1
2025-04-16 21:40:04    1
2025-04-16 21:11:54    1
2025-04-16 20:51:58    1
2025-04-16 15:43:09    1
2025-04-16 13:32:08    1
2025-04-16 13:17:25    1
2025-04-16 12:37:28    1
2025-04-16 12:27:44    1
2025-04-15 18:02:28    1
2025-04-15 17:54:22    1
2025-04-15 17:38:08    1
2025-04-15 17:29:19    1
2025-04-14 09:35:19    1
2025-04-1